In [1]:
import os
import re
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
alldata = []

In [3]:
# (using this as oppossed to native Pandas because txt doesn't injest easilly)
pdf_file = 'FormIIs.html'
pdf = open(pdf_file, "r")

In [4]:
# puts the data from the pdf into a list, skipping new line characters
for line in pdf:
    alldata.append(line.strip())

In [5]:
alldata

['<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"><HTML>',
 '<HEAD>',
 '<TITLE></TITLE>',
 '</HEAD>',
 '<BODY>',
 '<A name=1></a>American Land Title Association<br>',
 'ALTA Commitment<br>',
 'Adopted 6-17-06<br>',
 'CHICAGO TITLE INSURANCE COMPANY </span><span class="ft5">800-943-1196<br>',
 'SCHEDULE A<br>',
 'Loan No.:<br>',
 'Title No.: NC2<br>',
 '1.<br>',
 'Effective date: 9th day of March, 2017<br>',
 'Issue Date: 03/17/2017<br>',
 'This Title Insurance Commitment is good for 60 days from the effective date. If you close a transaction based on<br>',
 'this Title Insurance Commitment without a title update, Chicago Title Insurance Company will not issue a Title<br>',
 'Insurance Policy.<br>',
 '2.<br>',
 'Policy (or Policies) to be issued:<br>',
 '(a)<br>',
 "Owner's Policy 2006 Owner's Policy<br>",
 'Policy<br>',
 'TBD<br>',
 'Form<br>',
 'Amount<br>',
 'Proposed Insured:<br>',
 'Policy Premium:<br>',
 '(Additional discounts may apply)<br>',
 '(b)<br>',
 'Loan Pol

In [6]:
# makes a data frame out of the data 
raw_pdf = pd.DataFrame(alldata)

In [7]:
# finds the rows with blank data and fills them with Nan so they can be dropped, and drops them 
# (*note temporary commenting out because blank lines may be useful for grouping exceptions/requirements)
# raw_pdf = raw_pdf[raw_pdf != '']
# raw_pdf.dropna(inplace=True)

In [8]:
# lets reindex so that we don't have huge jumps in the index for dropped rows
raw_pdf.set_index(np.arange(0,len(raw_pdf)),inplace = True)

In [9]:
# creating the list of headings to hunt for, hard-cording from the pdf, and using pandas tricks to get the last line.
try:
    raw_pdf[raw_pdf[0].str.match('LEGAL DESCRIPTION')].index[0]
except:
    try:
        raw_pdf[raw_pdf[0].str.match('EXCEPTIONS FROM COVERAGE')].index[0]
    except:
        print "wut"
    else:
        headings = ['SCHEDULE A','EXCEPTIONS FROM COVERAGE','SCHEDULE C', 'SCHEDULE D' ,(raw_pdf[0][raw_pdf.index[-1]])]
else:
    headings = ['SCHEDULE A','LEGAL DESCRIPTION','REQUIREMENTS', 'EXCEPTIONS' , (raw_pdf[0][raw_pdf.index[-1]])]

In [10]:
# create a dataframe to store the index locations for the starting line of each section
locations = [0]
df_locations = pd.DataFrame(locations)

In [11]:
# run each heading in our list through a search, and place those start index locations into a dataframe
for heading in headings:
    df_locations[heading] = raw_pdf[raw_pdf[0].str.match(heading)].index[0]

In [12]:
# some column labeling and cleanup, and sorting by order they appear in the pdf
df_locations.drop(0,axis=1,inplace=True)
df_locations = df_locations.rename({0:'start_line'}).transpose()
df_locations.sort_values(by='start_line',inplace=True)

In [13]:
# shifting to a numeric index so it is easy to iterate over rows in Dataframe
df_locations['sections'] = df_locations.index
df_locations.reset_index(drop=True, inplace=True)

In [14]:
# creating the column indicating the ending line for each section of the pdf
end = df_locations['start_line'][1:len(df_locations)].values
end = np.append(end,df_locations['start_line'][len(df_locations)-1:len(df_locations)].values)
df_locations['end_line'] = end

In [15]:
df_locations

,start_line,sections,end_line
0,9,SCHEDULE A,85
1,85,LEGAL DESCRIPTION,98
2,98,REQUIREMENTS,196
3,196,EXCEPTIONS,277
4,277,</HTML>,277


In [16]:
# now we can use "start_line" and "end_line" to define slices of the raw_pdf Dataframe and make section-specific chunks
# and output results to csv
for index, row in df_locations.iterrows():
    section = str(row[1]).strip("</>")
    string = (pdf_file + "_" + "df" + "_" + section)  
    print string
    string = raw_pdf[row[0]: row[2]]
    string.to_csv(path_or_buf=(pdf_file + "_" + "df" + "_" + section + ".csv"))

FormIIs.html_df_SCHEDULE A
FormIIs.html_df_LEGAL DESCRIPTION
FormIIs.html_df_REQUIREMENTS
FormIIs.html_df_EXCEPTIONS
FormIIs.html_df_HTML
